In [1]:
text = "Hello, world! This is a tokenizer example."
tokens = text.split()  # 简单的分词方法，按空格分割
print(tokens)
# 输出: ['Hello,', 'world!', 'This', 'is', 'a', 'tokenizer', 'example.']


['Hello,', 'world!', 'This', 'is', 'a', 'tokenizer', 'example.']


In [2]:
vocab = {'Hello,': 1, 'world!': 2, 'This': 3, 'is': 4, 'a': 5, 'tokenizer': 6, 'example.': 7}
numericalized_tokens = [vocab[token] for token in tokens]
print(numericalized_tokens)
# 输出: [1, 2, 3, 4, 5, 6, 7]


[1, 2, 3, 4, 5, 6, 7]


In [1]:
import torch.nn as nn
import torch
# 假设词汇表大小为 8，嵌入维度为 3
embedding = nn.Embedding(10, 3)

# 将数值化后的 token 转换为嵌入向量
embedded_tokens = embedding(torch.tensor(numericalized_tokens))
print(embedded_tokens)
# 输出: tensor形状为 (7, 3) 的嵌入向量


NameError: name 'numericalized_tokens' is not defined

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.4.0


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

question = "What is the capital of France?"
context = "France is a country in Europe. It has a long history of cultural influence, especially in art and cuisine. Paris, the capital of France, is known for its landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. The city is also a global center for fashion and luxury goods, and it hosts several international organizations."

# 标记化输入并截断
inputs = tokenizer(question, context, truncation="only_second", max_length=50)
print(inputs)


{'input_ids': [101, 2054, 2003, 1996, 3007, 1997, 2605, 1029, 102, 2605, 2003, 1037, 2406, 1999, 2885, 1012, 2009, 2038, 1037, 2146, 2381, 1997, 3451, 3747, 1010, 2926, 1999, 2396, 1998, 12846, 1012, 3000, 1010, 1996, 3007, 1997, 2605, 1010, 2003, 2124, 2005, 2049, 16209, 2107, 2004, 1996, 1041, 13355, 2884, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [2]:
from paddlenlp.transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B", dtype="float16")
input_features = tokenizer("你好！请自我介绍一下。", return_tensors="pd")
outputs = model.generate(**input_features, max_length=128)
print(tokenizer.batch_decode(outputs[0]))

[2024-08-13 22:06:27,623] [    INFO] - We are using <class 'paddlenlp.transformers.qwen2.tokenizer.Qwen2Tokenizer'> to load 'Qwen/Qwen2-0.5B'.
[2024-08-13 22:06:28,041] [    INFO] - The `unk_token` parameter needs to be defined: we use `eos_token` by default.
[2024-08-13 22:06:28,256] [    INFO] - Adding <|im_start|> to the vocabulary
[2024-08-13 22:06:28,257] [    INFO] - Adding <|im_end|> to the vocabulary
[2024-08-13 22:06:28,257] [    INFO] - Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[2024-08-13 22:06:28,278] [    INFO] - We are using <class 'paddlenlp.transformers.qwen2.modeling.Qwen2ForCausalLM'> to load 'Qwen/Qwen2-0.5B'.
[2024-08-13 22:06:28,279] [    INFO] - Loading configuration file C:\Users\C1602\.paddlenlp\models\Qwen/Qwen2-0.5B\config.json
[2024-08-13 22:06:28,282] [    INFO] - Loading weights file from cache at C:\Users\C1602\.paddlenlp\models\Qwen/Qwen2-0.5B\model.safetensors


AttributeError: module 'mmap' has no attribute 'MAP_PRIVATE'